### Basic imports

In [1]:
import sys
import random
import numpy as np
import skmultilearn
import pandas as pd
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
import numpy as np
from sklearn import model_selection

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier

import spacy
# !pip install --user spacy
# !python3 -m spacy download pt_core_news_sm

### Load datasets

In [3]:
test_set = pd.read_csv('sets/lemmatized Set 0.csv', index_col=0)
set_1 = pd.read_csv('sets/lemmatized Set 1.csv', index_col=0)
set_2 = pd.read_csv('sets/lemmatized Set 2.csv', index_col=0)
set_3 = pd.read_csv('sets/lemmatized Set 3.csv', index_col=0)
set_4 = pd.read_csv('sets/lemmatized Set 4.csv', index_col=0)

# test_set = pd.read_csv('sets/Set 0.csv', index_col=0)
# set_1 = pd.read_csv('sets/Set 1.csv', index_col=0)
# set_2 = pd.read_csv('sets/Set 2.csv', index_col=0)
# set_3 = pd.read_csv('sets/Set 3.csv', index_col=0)
# set_4 = pd.read_csv('sets/Set 4.csv', index_col=0)


def string_to_array_serie(serie):
    return [[int(i) for i in t.replace("[", "").replace("]", "").split()] for t in serie]

test_set["Senado"] = string_to_array_serie(test_set["Senado"])
set_1["Senado"] = string_to_array_serie(set_1["Senado"])
set_2["Senado"] = string_to_array_serie(set_2["Senado"])
set_3["Senado"] = string_to_array_serie(set_3["Senado"])
set_4["Senado"] = string_to_array_serie(set_4["Senado"])

test_set["Referenda"] = string_to_array_serie(test_set["Referenda"])
set_1["Referenda"] = string_to_array_serie(set_1["Referenda"])
set_2["Referenda"] = string_to_array_serie(set_2["Referenda"])
set_3["Referenda"] = string_to_array_serie(set_3["Referenda"])
set_4["Referenda"] = string_to_array_serie(set_4["Referenda"])

In [4]:
train_sets = [set_1, set_2, set_3, set_4]

full_train = pd.concat(train_sets)
full_train

,Numero ato,Ementa + texto completo,Senado,Referenda
0,10095,dispor sobrar o comitê consultivo de nanotecno...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
1,10092,promulgar o protocolo de integração educativo ...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,10085,dispor sobrar o programar forçar o esporte seg...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0]"
3,10081,alterar o decretar n 8713 de 15 de abril de 20...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]"
4,10083,autorizar o empregar das forçar armar o garant...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...
1323,3336,dar novo redação aos arts 11 15 16 19 e 30 d...,"[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
1324,3330,dispor sobrar o redução do consumir de energia...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
1325,3338,aprovar o estruturar regimental e o quadrar de...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]"
1326,3328,alterar o decretar 2889 de 21 12 1998 que disp...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]"


### The following code can be used to lemmatize the datasets. Due to the high computational cost of applying lemmatization every time, we saved its result on new datasets, which are the already loaded in here.

In [18]:
# # Code for lemmatizing
# import matplotlib.pyplot as plt
# import numpy as np
# from keras.preprocessing.text import Tokenizer,  text_to_word_sequence

# from nltk import tokenize

# def lemmatize(texts, i, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     print(i)
#     texts_out = []
#     for sent in texts.split(" ")[:10000]:
#         doc=nlp(sent)
#         texts_out.append(" ".join([token.lemma_ for token in doc]))
#     return " ".join(texts_out)

# nlp = spacy.load('pt_core_news_sm')

# Test_X = [lemmatize(s, i) for (i, s) in enumerate(test_set["Ementa + texto completo"])]
# print("Test ok")

# # The following sets will be used for validation.
# set_1_X = [lemmatize(s, i) for (i, s) in enumerate(set_1["Ementa + texto completo"])]
# print("Set 1 ok")
# set_2_X = [lemmatize(s, i) for (i, s) in enumerate(set_2["Ementa + texto completo"])]
# print("Set 2 ok")
# set_3_X = [lemmatize(s, i) for (i, s) in enumerate(set_3["Ementa + texto completo"])]
# print("Set 3 ok")
# set_4_X = [lemmatize(s, i) for (i, s) in enumerate(set_4["Ementa + texto completo"])]
# print("Set 4 ok")

# # Save lemmatized sets for further use
# pd.DataFrame(Test_X).to_csv('sets/X lemmatized Set 0.csv')
# pd.DataFrame(set_1_X).to_csv('sets/X lemmatized Set 1.csv')
# pd.DataFrame(set_2_X).to_csv('sets/X lemmatized Set 2.csv')
# pd.DataFrame(set_3_X).to_csv('sets/X lemmatized Set 3.csv')
# pd.DataFrame(set_4_X).to_csv('sets/X lemmatized Set 4.csv')

# sets_X = [set_1_X, set_2_X, set_3_X, set_4_X]

# Train_X = pd.concat(sets_X)

### Creates train, test and validation X and Y sets to use in the model

In [5]:
Train_X = full_train["Ementa + texto completo"]
Test_X = test_set["Ementa + texto completo"]
set_1_X = set_1["Ementa + texto completo"]
set_2_X = set_2["Ementa + texto completo"]
set_3_X = set_3["Ementa + texto completo"]
set_4_X = set_4["Ementa + texto completo"]

In [6]:
Train_Y_sen = full_train["Senado"].values
Test_Y_sen = test_set["Senado"].values
Train_Y_minist = full_train["Referenda"].values
Test_Y_minist = test_set["Referenda"].values

set_1_Y_sen = set_1["Senado"].values
set_2_Y_sen = set_2["Senado"].values
set_3_Y_sen = set_3["Senado"].values
set_4_Y_sen = set_4["Senado"].values
sets_Y_sen = [set_1_Y_sen, set_2_Y_sen, set_3_Y_sen, set_4_Y_sen]

set_1_Y_minist = set_1["Referenda"].values
set_2_Y_minist = set_2["Referenda"].values
set_3_Y_minist = set_3["Referenda"].values
set_4_Y_minist = set_4["Referenda"].values
sets_Y_minist = [set_1_Y_minist, set_2_Y_minist, set_3_Y_minist, set_4_Y_minist]

In [7]:
Train_Y_sen = [[j for j in i] for i in Train_Y_sen]
Test_Y_sen = [[j for j in i] for i in Test_Y_sen]
Train_Y_minist = [[j for j in i] for i in Train_Y_minist]
Test_Y_minist = [[j for j in i] for i in Test_Y_minist]

### Remove stopwords

In [8]:
frequency_stopwords = pd.read_csv("stopwords/frequency_stopwords_list.csv", index_col=0)
frequency_stopwords = list(frequency_stopwords["0"].values)

In [9]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def removeStopWords(texts, st= stopwords.words('portuguese')):
    texts_r = []
    for t in texts:
        final_t = ""
        for w in t.split(" "):
            if w not in st and w not in frequency_stopwords and w!=".":
                final_t += w + " "
        texts_r.append(final_t)
    return texts_r

Train_X = removeStopWords(Train_X)
Test_X = removeStopWords(Test_X)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/caiocampos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Get Tf-Idf representations for the texts

In [10]:
Tfidf_vect = TfidfVectorizer()
Tfidf_vect.fit(Train_X)

Train_X = Tfidf_vect.transform(Train_X)
Test_X = Tfidf_vect.transform(Test_X)

sets_X = [set_1_X, set_2_X, set_3_X, set_4_X]

In [11]:
sen_classes_names = ["Saúde", "Relações Exteriores", "Meio ambiente", "Educação, cultura e esporte", "Segurança Pública e Defesa", "Trabalho e Previdência", "Agricultura, pecuária e pesca", "Ciência, tecnologia e comunicações", "Social", "Indústria, comércio, turismo, transporte/transporte de mercadorias", "Economia, planejamento e sistema financeiro", "Assuntos internos,  cargos e comissões, Estado", "Tributos", "Minas e Energia", "Justiça e direitos"]
minist_classes_names = ["Saúde", "Relações Exteriores", "Meio ambiente", "Educação, cultura e esporte", "Justiça e Segurança", "Trabalho e Previdência", "Transporte/transporte de mercadorias", "Agricultura, pecuária e pesca", "Ciência e tecnologia", "Social", "Presidência", "Economia e planejamento", "Indústria, comércio, obras públicas, turismo"]
stdout = sys.stdout

## Special metrics

In [16]:
def acerto_absoluto(test_y_sen, test_pred, number_labels):
    acerto_100 = 0
    test_prediction_binary = test_pred
    for i in range(len(test_prediction_binary)):
        acertos_internos = 0
        for j in range(len(test_prediction_binary[i])):
            if test_y_sen[i][j] == test_prediction_binary[i][j]:
                acertos_internos+=1
        if acertos_internos == number_labels:
            acerto_100+=1
    return acerto_100/len(test_prediction_binary)*100

### Generates validation csv's in order to save the whole validation procedure and results

In [12]:
def generate_validation_csv(f1s_per_class, precision_per_class, recall_per_class, prefix, names):
    final_metrics = (f1s_per_class, np.array([np.mean(f1s_per_class, axis=0)]), np.array([np.std(f1s_per_class, axis=0, ddof=1)]),
                     precision_per_class, np.array([np.mean(precision_per_class, axis=0)]), np.array([np.std(precision_per_class, axis=0, ddof=1)]),
                     recall_per_class, np.array([np.mean(recall_per_class, axis=0)]), np.array([np.std(recall_per_class, axis=0, ddof=1)]) )
    f1s_per_class_pd = pd.DataFrame(np.concatenate(final_metrics, axis=0))
    f1s_per_class_pd.columns = names
    f1s_per_class_pd.index=["validation set 1 f1", "validation set 2 f1", "validation set 3 f1", "validation set 4 f1", "validation sets mean f1", "validation std f1",
                            "validation set 1 precision", "validation set 2 precision", "validation set 3 precision", "validation set 4 precision", "validation sets mean precision", "validation std precision",
                            "validation set 1 recall", "validation set 2 recall", "validation set 3 recall", "validation set 4 recall", "validation sets mean recall", "validation std recall"]
    f1s_per_class_pd.to_csv("../optimization/validation_results/"+prefix+"iteration_"+str(iterations)+".csv", index=True, header=True)

### Get hyperparameters through cross-validation for the Senate-based classification

In [50]:
import os
os.chdir("/home/caiocampos/andre/radar_wisemap")
sys.path.append('hypermapper/scripts/')
import hypermapper

iterations = 0

def optimize_SVM(parameters):
    global iterations
    f1s = 0
    f1s_per_class = []
    precision_per_class = []
    recall_per_class = []
    
    for i in range(len(train_sets)):        
        sets_index = []
        for t in range(len(train_sets)):
            if t!=i:
                sets_index.append(t)

        train_X = Tfidf_vect.transform(np.concatenate(np.array(sets_X, dtype=object)[sets_index]))
        validation_X = Tfidf_vect.transform(sets_X[i])

        train_Y_sen = np.concatenate(np.array(sets_Y_sen, dtype=object)[sets_index])
        train_Y_sen = np.array([np.array(t) for t in train_Y_sen])
        validation_Y_sen = sets_Y_sen[i]
        validation_Y_sen = np.array([np.array(v) for v in validation_Y_sen])

        sv = OneVsRestClassifier(svm.SVC(C=parameters['regularization'], kernel='linear', gamma='auto', random_state=42))
        sv.fit(train_X, np.array(train_Y_sen))
        
        validation_prediction = sv.predict(validation_X)

        f1s+=f1_score(y_true=validation_Y_sen, y_pred=validation_prediction, average='macro')
        f1s_per_class.append(f1_score(y_true=validation_Y_sen, y_pred=validation_prediction, average=None))
        precision_per_class.append(precision_score(y_true=validation_Y_sen, y_pred=validation_prediction, average=None))
        recall_per_class.append(recall_score(y_true=validation_Y_sen, y_pred=validation_prediction, average=None))
    generate_validation_csv(f1s_per_class, precision_per_class, recall_per_class, "SVM_sen_", sen_classes_names)
    f1_loss=1-f1s/4
    print("Iteration "+str(iterations)+":\nC: ", parameters['regularization'], " || (1 - macro_F1): ", f1_loss)
    iterations+=1
    return f1_loss

os.chdir("hypermapper/")
stdout = sys.stdout
print(os.getcwd())
hypermapper.optimize("../optimization/SVM_scenario.json", optimize_SVM)
os.chdir("/home/caiocampos/andre/radar_wisemap/")
sys.stdout = stdout

/home/caiocampos/andre/radar_wisemap/hypermapper
Design of experiment phase, number of doe samples = 7 .......
Iteration 0:
C:  65.787422523204  || (1 - macro_F1):  0.235262060069236
Iteration 1:
C:  28.211968046088558  || (1 - macro_F1):  0.23307659489736288
Iteration 2:
C:  76.89208922944057  || (1 - macro_F1):  0.23548003807681694
Iteration 3:
C:  0.9588314481920487  || (1 - macro_F1):  0.3158139881335934
Iteration 4:
C:  4.279203942309014  || (1 - macro_F1):  0.23488531725448292
Iteration 5:
C:  13.356845211160689  || (1 - macro_F1):  0.23151176042009025
Iteration 6:
C:  1  || (1 - macro_F1):  0.3076747744997087
regularization,1 - F1,Timestamp
65.787422523204,0.235262060069236,199408
28.211968046088558,0.23307659489736288,400991
76.89208922944057,0.23548003807681694,599430
0.9588314481920487,0.3158139881335934,811872
4.279203942309014,0.23488531725448292,1022257
13.356845211160689,0.23151176042009025,1224946
1,0.3076747744997087,1437811


End of doe phase, the number of new configu

### Train the final Senate model with the best hyperparameters

In [24]:
optimization_results = pd.read_csv("optimization/sen_svm_validation_hypermapper_output.csv")
best_parameters_idx = optimization_results["1 - F1"].idxmin()
parameters = pd.DataFrame(optimization_results.iloc[best_parameters_idx]).transpose()
parameters

,regularization,1 - F1,Timestamp
15,15.397598,0.230697,3281619.0


In [25]:
validation_results = pd.read_csv("optimization/validation_results/SVM_sen_iteration_"+str(best_parameters_idx)+".csv", index_col=0)
validation_results

,Saúde,Relações Exteriores,Meio ambiente,"Educação, cultura e esporte",Segurança Pública e Defesa,Trabalho e Previdência,"Agricultura, pecuária e pesca","Ciência, tecnologia e comunicações",Social,"Indústria, comércio, turismo, transporte/transporte de mercadorias","Economia, planejamento e sistema financeiro","Assuntos internos, cargos e comissões, Estado",Tributos,Minas e Energia,Justiça e direitos
validation set 1 f1,0.500000,0.968966,0.633333,0.813187,0.824121,0.790698,0.740741,0.709677,0.694737,0.607843,0.784615,0.877949,0.876543,0.714286,0.630137
validation set 2 f1,0.590909,0.966957,0.711864,0.835165,0.835821,0.897959,0.761905,0.774194,0.753927,0.642202,0.737255,0.855638,0.884848,0.880000,0.750000
validation set 3 f1,0.536585,0.975862,0.656716,0.727273,0.840000,0.867925,0.767442,0.793651,0.697436,0.634615,0.781250,0.884735,0.900585,0.861111,0.589744
validation set 4 f1,0.625000,0.966957,0.754098,0.816327,0.828283,0.808511,0.708861,0.727273,0.713514,0.697248,0.757812,0.868687,0.905882,0.816901,0.602410
validation sets mean f1,0.563124,0.969685,0.689003,0.797988,0.832056,0.841273,0.744737,0.751199,0.714903,0.645477,0.765233,0.871752,0.891965,0.818075,0.643073
validation std f1,0.055646,0.004225,0.054472,0.048132,0.007176,0.050183,0.026541,0.039274,0.027304,0.037529,0.022136,0.012597,0.013619,0.074073,0.073254
validation set 1 precision,0.714286,0.979094,0.760000,0.902439,0.881720,0.894737,0.882353,0.846154,0.750000,0.738095,0.836066,0.875256,0.946667,0.862069,0.884615
validation set 2 precision,0.619048,0.989324,0.807692,0.950000,0.875000,0.956522,0.842105,0.857143,0.827586,0.744681,0.803419,0.882609,0.924051,0.916667,0.825000
validation set 3 precision,0.785714,0.989510,0.758621,0.842105,0.903226,0.821429,0.868421,0.862069,0.800000,0.846154,0.840336,0.893082,0.905882,0.939394,0.741935
validation set 4 precision,0.750000,0.985816,0.884615,0.888889,0.901099,0.826087,0.875000,0.800000,0.785714,0.826087,0.808333,0.877551,0.962500,0.935484,0.714286


In [14]:
sv = OneVsRestClassifier(svm.SVC(C=optimization_results.iloc[best_parameters_idx]["regularization"], kernel='linear', gamma='auto', random_state=42))
sv.fit(Train_X, np.array(Train_Y_sen))
predictions_svm = sv.predict(Test_X)

print(classification_report(np.array(Test_Y_sen), predictions_svm, digits=3, target_names=sen_classes_names))

                                                                    precision    recall  f1-score   support

                                                             Saúde      0.679     0.633     0.655        30
                                               Relações Exteriores      0.993     0.963     0.978       294
                                                     Meio ambiente      0.862     0.735     0.794        34
                                       Educação, cultura e esporte      0.861     0.608     0.713        51
                                        Segurança Pública e Defesa      0.821     0.885     0.852       104
                                            Trabalho e Previdência      0.957     0.759     0.846        29
                                     Agricultura, pecuária e pesca      0.865     0.727     0.790        44
                                Ciência, tecnologia e comunicações      0.875     0.848     0.862        33
                           

/home/caiocampos/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
pd.DataFrame(predictions_svm).to_csv("test_results/SVM_sen.csv", index=None, header=None)

In [17]:
for i in range (len(Test_Y_sen[0])+1):
    print(str(i)+"/"+str(len(Test_Y_sen[0]))+" classes:", acerto_absoluto(Test_Y_sen, predictions_svm, i))

0/15 classes: 0.0
1/15 classes: 0.0
2/15 classes: 0.0
3/15 classes: 0.0
4/15 classes: 0.0
5/15 classes: 0.0
6/15 classes: 0.0
7/15 classes: 0.0
8/15 classes: 0.0
9/15 classes: 0.0
10/15 classes: 0.0
11/15 classes: 0.07457121551081282
12/15 classes: 0.8202833706189411
13/15 classes: 7.38255033557047
14/15 classes: 15.659955257270694
15/15 classes: 76.0626398210291


### Get hyperparameters through cross-validation for the "Referenda"-based classification

In [19]:
import os
os.chdir("/home/caiocampos/andre/radar_wisemap")
sys.path.append('hypermapper/scripts/')
import hypermapper

iterations = 0

def optimize_SVM_minist(parameters):
    global iterations
    f1s = 0
    f1s_per_class = []
    precision_per_class = []
    recall_per_class = []
    
    for i in range(len(train_sets)):        
        sets_index = []
        for t in range(len(train_sets)):
            if t!=i:
                sets_index.append(t)

        train_X = Tfidf_vect.transform(np.concatenate(np.array(sets_X, dtype=object)[sets_index]))
        validation_X = Tfidf_vect.transform(sets_X[i])

        train_Y_minist = np.concatenate(np.array(sets_Y_minist, dtype=object)[sets_index])
        train_Y_minist = np.array([np.array(t) for t in train_Y_minist])
        validation_Y_minist = sets_Y_minist[i]
        validation_Y_minist = np.array([np.array(v) for v in validation_Y_minist])

        sv = OneVsRestClassifier(svm.SVC(C=parameters['regularization'], kernel='linear', gamma='auto', random_state=42))
        sv.fit(train_X, np.array(train_Y_minist))
        
        validation_prediction = sv.predict(validation_X)

        f1s+=f1_score(y_true=validation_Y_minist, y_pred=validation_prediction, average='macro')
        f1s_per_class.append(f1_score(y_true=validation_Y_minist, y_pred=validation_prediction, average=None))
        precision_per_class.append(precision_score(y_true=validation_Y_minist, y_pred=validation_prediction, average=None))
        recall_per_class.append(recall_score(y_true=validation_Y_minist, y_pred=validation_prediction, average=None))
    generate_validation_csv(f1s_per_class, precision_per_class, recall_per_class, "SVM_minist_", minist_classes_names)
    f1_loss=1-f1s/4
    print("Iteration "+str(iterations)+":\nC: ", parameters['regularization'], " || (1 - macro_F1): ", f1_loss)
    iterations+=1
    return f1_loss

os.chdir("hypermapper/")
stdout = sys.stdout
print(os.getcwd())
hypermapper.optimize("../optimization/SVM_scenario.json", optimize_SVM_minist)
os.chdir("/home/caiocampos/andre/radar_wisemap/")
sys.stdout = stdout

/home/caiocampos/andre/radar_wisemap/hypermapper
Design of experiment phase, number of doe samples = 7 .......
Iteration 0:
C:  74.86325264378715  || (1 - macro_F1):  0.2663617368635687
Iteration 1:
C:  78.8225228102691  || (1 - macro_F1):  0.2663600524299147
Iteration 2:
C:  46.65617652285129  || (1 - macro_F1):  0.26661123421886757
Iteration 3:
C:  15.416443782065201  || (1 - macro_F1):  0.26321822436341435
Iteration 4:
C:  40.65825485876155  || (1 - macro_F1):  0.26591799175954156
Iteration 5:
C:  60.52479099109636  || (1 - macro_F1):  0.26594586107827023
Iteration 6:
C:  1  || (1 - macro_F1):  0.28811013150706966
regularization,1 - F1,Timestamp
74.86325264378715,0.2663617368635687,218222
78.8225228102691,0.2663600524299147,436399
46.65617652285129,0.26661123421886757,657217
15.416443782065201,0.26321822436341435,884254
40.65825485876155,0.26591799175954156,1105299
60.52479099109636,0.26594586107827023,1324497
1,0.28811013150706966,1570250


End of doe phase, the number of new confi

### Train the final "Referenda" model with the best hyperparameters

In [18]:
optimization_results_minist = pd.read_csv("optimization/minist_svm_validation_hypermapper_output.csv")
best_parameters_idx_minist = optimization_results_minist["1 - F1"].idxmin()
parameters_minist = pd.DataFrame(optimization_results_minist.iloc[best_parameters_idx_minist]).transpose()
parameters_minist

,regularization,1 - F1,Timestamp
34,7.912664,0.258986,7916418.0


In [19]:
validation_results_minist = pd.read_csv("optimization/validation_results/SVM_minist_iteration_"+str(best_parameters_idx)+".csv", index_col=0)
validation_results_minist

,Saúde,Relações Exteriores,Meio ambiente,"Educação, cultura e esporte",Justiça e Segurança,Trabalho e Previdência,Transporte/transporte de mercadorias,"Agricultura, pecuária e pesca",Ciência e tecnologia,Social,Presidência,Economia e planejamento,"Indústria, comércio, obras públicas, turismo"
validation set 1 f1,0.656716,0.948247,0.747253,0.862275,0.783099,0.737864,0.655172,0.758621,0.747475,0.656716,0.455927,0.880249,0.716129
validation set 2 f1,0.676471,0.951261,0.772727,0.840237,0.801205,0.678899,0.615385,0.630435,0.713043,0.683333,0.549133,0.882812,0.690476
validation set 3 f1,0.734177,0.955932,0.853933,0.800000,0.779661,0.719298,0.625000,0.626506,0.735849,0.666667,0.491018,0.871473,0.662500
validation set 4 f1,0.693333,0.949495,0.760870,0.817073,0.770588,0.754386,0.655172,0.673913,0.750000,0.661538,0.520468,0.869697,0.689873
validation sets mean f1,0.690174,0.951234,0.783696,0.829896,0.783638,0.722612,0.637682,0.672369,0.736592,0.667064,0.504136,0.876058,0.689745
validation std f1,0.032932,0.003367,0.047968,0.027163,0.012845,0.032476,0.020574,0.061383,0.016865,0.011582,0.039948,0.006446,0.021901
validation set 1 precision,0.814815,0.959459,0.809524,0.947368,0.837349,0.844444,0.826087,0.891892,0.973684,0.721311,0.513699,0.885759,0.781690
validation set 2 precision,0.851852,0.975862,0.918919,0.898734,0.892617,0.740000,0.761905,0.805556,0.788462,0.820000,0.582822,0.895404,0.711656
validation set 3 precision,0.725000,0.982578,0.904762,0.914286,0.831325,0.788462,0.714286,0.838710,0.847826,0.703125,0.539474,0.886762,0.741259
validation set 4 precision,0.812500,0.965753,0.833333,0.893333,0.867550,0.843137,0.703704,0.775000,0.829787,0.741379,0.566879,0.860570,0.741497


In [17]:
sv = OneVsRestClassifier(svm.SVC(C=optimization_results_minist.iloc[best_parameters_idx]["regularization"], kernel='linear', gamma='auto', random_state=42))
sv.fit(Train_X, np.array(Train_Y_minist))
predictions_svm = sv.predict(Test_X)

print(classification_report(np.array(Test_Y_minist), predictions_svm, digits=3, target_names=minist_classes_names))

                                              precision    recall  f1-score   support

                                       Saúde      0.800     0.667     0.727        42
                         Relações Exteriores      0.960     0.942     0.951       308
                               Meio ambiente      0.886     0.674     0.765        46
                 Educação, cultura e esporte      0.903     0.730     0.807        89
                         Justiça e Segurança      0.780     0.738     0.758       187
                      Trabalho e Previdência      0.714     0.593     0.648        59
        Transporte/transporte de mercadorias      0.783     0.600     0.679        30
               Agricultura, pecuária e pesca      0.854     0.648     0.737        54
                        Ciência e tecnologia      0.764     0.700     0.730        60
                                      Social      0.708     0.718     0.713        71
                                 Presidência      0.4

/home/caiocampos/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
pd.DataFrame(predictions_svm).to_csv("test_results/SVM_minist.csv", index=None, header=None)

In [21]:
for i in range (len(Test_Y_minist[0])+1):
    print(str(i)+"/"+str(len(Test_Y_minist[0]))+" classes:", acerto_absoluto(Test_Y_sen, predictions_svm, i))

0/13 classes: 0.0
1/13 classes: 0.0
2/13 classes: 0.0
3/13 classes: 0.0
4/13 classes: 0.0
5/13 classes: 0.0
6/13 classes: 0.07457121551081282
7/13 classes: 0.07457121551081282
8/13 classes: 0.37285607755406414
9/13 classes: 3.206562266964952
10/13 classes: 8.650260999254288
11/13 classes: 25.205070842654738
12/13 classes: 29.30648769574944
13/13 classes: 33.1096196868009
